In [ ]:
import Pkg 
Pkg.activate("./..")

In [ ]:
import QuantumGrav as QG
import Flux 
import StructArrays as SA
import CausalSets

In [ ]:
equal_data = [QG.DataGeneration.generateDataForManifold(
        ; dimension = 2,
        manifoldname = m,
        seed = 329478,
        num_datapoints = 256,
        equal_size = true,
        size_distr = d -> CausalSets.Uniform(0.7 * 10^(d + 1), 1.1 * 10^(d + 1)),
        make_diamond = d->CausalSets.CausalDiamondBoundary{d}(1.0),
        make_box = d->CausalSets.BoxBoundary{d}((
            ([-0.49 for i in 1:d]...,), ([0.49 for i in 1:d]...,)))
) for m in ["minkowski", "deSitter", "antiDeSitter"]]

In [ ]:
SA.StructArray(equal_data[1])

In [ ]:
cat([ for d in equal_data]...; dims = 2)